In [2]:
import os


from dotenv import load_dotenv
#set the env values just outside of virtual environment
custom_env_path = lambda a : '/'.join(os.get_exec_path()[0].split('\\')[:-2])+'/'+a
load_dotenv(custom_env_path('chatter.env'))

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
## LangSmith tracking
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

## Data Ingestion

In [3]:
## Data Ingestion
from langchain_community.document_loaders import TextLoader
loader = TextLoader("speech.txt")
text_document = loader.load()
text_document

[Document(page_content="Five score years ago, a great American, in whose symbolic shadow we stand today, signed the Emancipation Proclamation. This momentous decree came as a great beacon light of hope to millions of Negro slaves who had been seared in the flames of withering injustice. It came as a joyous daybreak to end the long night of their captivity.\n\n\nBut 100 years later, the Negro still is not free. One hundred years later, the life of the Negro is still sadly crippled by the manacles of segregation and the chains of discrimination. One hundred years later, the Negro lives on a lonely island of poverty in the midst of a vast ocean of material prosperity. One hundred years later the Negro is still languished in the corners of American society and finds himself in exile in his own land. And so we've come here today to dramatize a shameful condition. In a sense we've come to our nation's capital to cash a check.\n\n\nWhen the architects of our republic wrote the magnificent wor

In [4]:
# web based loader
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                       bs_kwargs= dict(parse_only = bs4.SoupStrainer(
                           class_ = ("post-title","post-content","post-header")
                       )),)
web_document = loader.load()
web_document

[Document(page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final re

In [7]:

from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('Turning Data into Action.pdf')
pdf_document = loader.load()
pdf_document

[Document(page_content='A Capital Services White Paper by Dr. Alfred FurthT urning Data into Action: How Credit Card \nPrograms Can Benefit from the World of Big Data\n  Introduction\nScientists tell us that enough sunlight falls on the Earth’s surface every hour to meet world energy demand for \nan entire year. But all that sunlight does little good from a power-producing standpoint if we can’t harness it \nand convert it to usable energy.\nThe same is true of data. As new sources of data emerge and the costs of storing the data continue to \ndecrease, the challenge is how to make that data useful; how to sift the most relevant information out of the \ndaily tsunami of data streams, analyze it, and make it a tool for profitability. \nWhen it comes to credit card programs, their level of success and profitability is often directly related to how \nwell the bank or credit union takes advantage of the data available to them through their commitment to \nanalytics. The combination of anal

## Transform

In [8]:
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)
documents = text_splitter.split_documents(pdf_document)
documents[:5]

[Document(page_content='A Capital Services White Paper by Dr. Alfred FurthT urning Data into Action: How Credit Card \nPrograms Can Benefit from the World of Big Data\n  Introduction\nScientists tell us that enough sunlight falls on the Earth’s surface every hour to meet world energy demand for \nan entire year. But all that sunlight does little good from a power-producing standpoint if we can’t harness it \nand convert it to usable energy.\nThe same is true of data. As new sources of data emerge and the costs of storing the data continue to \ndecrease, the challenge is how to make that data useful; how to sift the most relevant information out of the \ndaily tsunami of data streams, analyze it, and make it a tool for profitability. \nWhen it comes to credit card programs, their level of success and profitability is often directly related to how \nwell the bank or credit union takes advantage of the data available to them through their commitment to', metadata={'source': 'Turning Data 

## Embed

In [10]:
# using OpenAI embeddings
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db_c = Chroma.from_documents(documents[:5],OpenAIEmbeddings())

d:\Study2\git\Environments\Chatter\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
from langchain_community.vectorstores import FAISS
db_f = FAISS.from_documents(documents[:5],OpenAIEmbeddings())